In [1]:
import numpy as np

In [ ]:
def sample_paths(S_0, T, mu, sigma, lamb, Y, I):
    X_0 = np.log(S_0)
    dt = T / I

    for i in range(I):
        Z = np.random.standard_normal()
        N = np.random.poisson(lamb)
        Y = np.exp(np.normal(0,1,N))

        if N == 0:
            M = 0
        else:
            M = np.sum(Y)

        for t in range(1,T):
            X[t] = X[t-1] + (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z + np.log(M)
            S[t] = np.exp(X[t])

    return S

        

In [22]:
import numpy as np

def sample_paths(S_0, T, mu, sigma, lamb, Y_mean, I, N):
    dt = T / I
    X = np.zeros((I, T))
    S = np.zeros((I, T))

    for i in range(I):
        S[i, 0] = S_0
        X[i, 0] = np.log(S_0)
        
        for t in range(1, T):
            Z = np.random.standard_normal()
            Jumps = np.random.poisson(lamb * dt)
            Y = np.exp(np.random.normal(Y_mean, 1, Jumps))

            if Jumps == 0:
                M = 0
            else:
                M = np.sum(Y)

            X[i, t] = X[i, t-1] + (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z + np.log(1 + M)
            S[i, t] = np.exp(X[i, t])

    return S


In [24]:
output = sample_paths(100, 10, 0.05, 0.2, 0.5, 1, 5, 5)
print(output)

[[1.00000000e+02 1.02038628e+02 4.28174287e+03 2.19060357e+04
  2.24401307e+05 2.66289225e+05 1.62812958e+06 7.05797525e+06
  1.13406365e+07 3.30954529e+07]
 [1.00000000e+02 1.70910000e+02 2.18528499e+03 1.31505852e+04
  4.08395503e+05 2.83435772e+05 1.99680132e+06 2.55771569e+06
  5.58580377e+07 1.20134750e+08]
 [1.00000000e+02 1.71977004e+03 4.75812346e+03 2.51667654e+04
  3.51123962e+04 7.33079315e+04 2.07148116e+05 2.51491045e+05
  2.21953148e+05 4.29826463e+05]
 [1.00000000e+02 1.65961065e+02 2.49836082e+02 3.65416810e+02
  1.55182588e+03 1.73235865e+03 6.33188799e+03 7.34032307e+04
  8.11544569e+04 1.67592437e+05]
 [1.00000000e+02 2.84018460e+02 3.18360225e+02 1.08310130e+03
  1.01405565e+03 2.68762744e+03 2.34737938e+04 1.82608104e+05
  4.55255457e+05 4.63990239e+05]]


In [4]:
def lognormal_Y(mu, sigma_y):
    """
    Returns a sample from a log-normal distribution.
    
    Parameters:
    mu : float : Mean of the logarithm of the variable
    sigma_y : float : Standard deviation of the logarithm of the variable
    
    Returns:
    float : Sampled value
    """
    return np.random.lognormal(mu, sigma_y)

def sample_paths_ln_lognormal(S0, T, sigma, lambda_, mu, sigma_y, I):
    """
    Jump-diffusion Sample Paths LN with Y distributed log-normally (Algorithm 1 modified)
    
    Parameters:
    S0 : float : Initial stock price
    T : float : Time horizon
    sigma : float : Volatility
    lambda_ : float : Jump-intensity
    mu : float : Mean of the logarithm of Y
    sigma_y : float : Standard deviation of the logarithm of Y
    I : int : Number of sample paths
    
    Returns:
    list : Sampled paths
    """
    
    dt = T / I
    S = np.zeros(I)
    X = np.log(S0)
    
    for i in range(I):
        Z = np.random.normal(0, 1)
        N = np.random.poisson(lambda_ * dt)
        
        if N == 0:
            M = 0
        else:
            M = sum([np.log(lognormal_Y(mu, sigma_y)) for _ in range(N)])
        
        X += (sigma**2 / 2) * dt + sigma * np.sqrt(dt) * Z + M
        S[i] = np.exp(X)
    
    return S

In [25]:
import numpy as np

def merton_jump_paths(S, T, r, sigma, lam, m, v, steps, Npaths):
    """
    Simulate sample paths using the Merton jump-diffusion model.
    
    Parameters:
    S : float : Initial stock price
    T : float : Time horizon
    r : float : Risk-free rate
    sigma : float : Volatility of the continuous component
    lam : float : Jump intensity
    m : float : Expected jump size
    v : float : Volatility of jump
    steps : int : Number of time steps
    Npaths : int : Number of sample paths
    
    Returns:
    np.array : Matrix of sample paths over time
    """
    size = (steps, Npaths)
    dt = T / steps
    poi_rv = np.multiply(np.random.poisson(lam * dt, size=size),
                         np.random.normal(m, v, size=size)).cumsum(axis=0)
    geo = np.cumsum(((r - sigma**2/2 - lam*(m + v**2*0.5))*dt +
                     sigma*np.sqrt(dt) *
                     np.random.normal(size=size)), axis=0)
    
    return np.exp(geo + poi_rv) * S

# Simulate 5 sample paths
sample_paths_matrix = merton_jump_paths(100, 1, 0.05, 0.2, 0.5, 0, 0.1, 100, 5)
sample_paths_matrix


array([[ 98.0847969 ,  99.65406594, 102.73407344, 101.99414528,
         94.54912262],
       [ 98.73915933,  96.31307118, 105.90430658, 102.58885653,
         95.57065451],
       [ 95.48641533,  95.60265202, 106.20152834, 101.57683263,
         96.27309963],
       [ 94.38747262,  94.43361284, 107.24602755,  99.99580694,
         97.06640292],
       [ 93.64721308,  91.44553884, 105.79116493, 100.79153285,
         99.55220011],
       [ 95.6879904 ,  92.68763441, 102.67018874, 101.96060539,
         99.32238167],
       [ 95.58075513,  94.7226304 , 105.23055465, 100.74352636,
         99.6323481 ],
       [ 91.73288335,  96.50174191, 107.06245591, 102.43893805,
        100.28777793],
       [ 92.39219884,  95.00092009, 105.13029046, 105.49103879,
        100.82488288],
       [ 91.88765752,  96.25173458, 105.4275713 , 109.39719515,
         99.15025552],
       [ 93.74449191,  95.77367065, 106.3616083 , 108.59227016,
         99.90772396],
       [ 95.2647504 ,  95.50551749, 107.189